In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import os
import neurokit2 as nk
from concurrent.futures import ThreadPoolExecutor
from scipy.signal import butter, lfilter
from tqdm import tqdm

In [2]:
from data_loader_mat import load_data

main_directory = r'E:\Coding\Jupyter_files\ECG_2\Data_saves\Reshaped_data_length'
loaded_data = load_data(main_directory)

Data is loading...: 100%|███████████████████████████████████████████████████████| 24246/24246 [01:22<00:00, 294.73it/s]


In [3]:
def bandpass_filter(signal, lowcut, highcut, sampling_rate, order=2):
    nyquist = 0.5 * sampling_rate
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, signal)
    return y

In [ ]:
#DO NOT USE THIS PIECE OF CODE

# Progress bar setup:

total_files = len(loaded_data)
progress_bar = tqdm(total = total_files, desc = 'Filtering is in process...')


# Bandpass filter parameters:

lowcut = 0.5
highcut = 24.0
sampling_rate = 500

filtered_data = {}

# Applying Bandpass filter:

for filepath, data in loaded_data.items():
    if filepath.endswith('.mat'):
        ecg_signal = data['val'].flatten()
        filtered_signal = bandpass_filter(ecg_signal,lowcut,highcut,sampling_rate)
        filtered_data[filepath] = filtered_signal
    progress_bar.update(1)
progress_bar.close()

In [4]:
#PRESERVE THE ORIGINAL SHAPE AND APPLY THE BANDPASS FILTER

# Progress bar setup
total_files = len(loaded_data)
progress_bar = tqdm(total=total_files, desc='Filtering is in process...')

# Bandpass filter parameters
lowcut = 0.5
highcut = 24.0
sampling_rate = 500

filtered_data = {}

# Applying Bandpass filter
for filepath, data in loaded_data.items():
    if 'val' in data:
        ecg_signal = data['val'].flatten()  # Flatten the original signal
        filtered_signal = bandpass_filter(ecg_signal, lowcut, highcut, sampling_rate)
        
        # Preserve the 'val' key and shape (12, 5000)
        filtered_data[filepath] = {'val': filtered_signal.reshape(12, 5000)}  # Reshape to original shape
        
    progress_bar.update(1)

progress_bar.close()

Filtering is in process...: 100%|███████████████████████████████████████████████| 24246/24246 [00:29<00:00, 829.43it/s]


In [ ]:
from data_saver import data_saver_v1
output_directory = r'E:\Coding\Jupyter_files\ECG_2\Data_saves\Filtered_data'
data_saver_v1(filtered_data, output_directory)